In [ ]:
"""
=============================
OOB Errors for Random Forests
=============================

The ``RandomForestClassifier`` is trained using *bootstrap aggregation*, where
each new tree is fit from a bootstrap sample of the training observations
:math:`z_i = (x_i, y_i)`. The *out-of-bag* (OOB) error is the average error for
each :math:`z_i` calculated using predictions from the trees that do not
contain :math:`z_i` in their respective bootstrap sample. This allows the
``RandomForestClassifier`` to be fit and validated whilst being trained [1]_.

The example below demonstrates how the OOB error can be measured at the
addition of each new tree during training. The resulting plot allows a
practitioner to approximate a suitable value of ``n_estimators`` at which the
error stabilizes.
"""

In [ ]:
from collections import OrderedDict
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

In [ ]:
"""
Cuando se trabaja con algoritmos de aprendizaje automático que implican aleatoriedad, como entrenar un modelo 
de Bosques Aleatorios, hay que tener en cuenta que los resultados pueden variar dependiendo de la semilla aleatoria que usemos. 
La semilla aleatoria determina la secuencia de números pseudoaleatorios generados por el algoritmo.

Al establecer una semilla aleatoria específica, se asegura que los resultados sean reproducibles, es decir, 
que al ejecutar el código varias veces con la misma semilla aleatoria, se obtendrán los mismos resultados cada vez. 
Esto es útil para fines de depuración, experimentación y comparación de modelos.

Esto garantiza que, si el código se ejecuta varias veces con la misma semilla aleatoria, se obtendrán los mismos 
resultados en cada ejecución, lo que facilita la reproducibilidad de los experimentos y los resultados del modelo.
"""

""" 
Se define una semilla aleatoria (RANDOM_STATE) para reproducibilidad. 
"""


In [ ]:
RANDOM_STATE = 123
# se utiliza para asegurar que se obtenga el mismo conjunto de datos si se ejecuta el código varias veces.

In [ ]:
""" 
Se genera un conjunto de datos de clasificación binaria utilizando make_classification de scikit-learn. 

La función make_classification devuelve dos matrices NumPy: 'X', que contiene las características de las muestras generadas, 
e 'y', que contiene las etiquetas de clase asociadas a cada muestra. 
"""

X, y = make_classification(
    n_samples=500,  # número total de muestras
    n_features=25,  # número de características (o variables) que tendrá cada muestra
    n_clusters_per_class=1,   # número de clústeres por clase. Hace que las muestras de cada clase se agrupen en un solo grupo.
    n_informative=15,  # número de características que son relevantes (informativas) para predecir la variable objetivo
                       # De las 25 características totales, 15 serán informativas,
    random_state=RANDOM_STATE,  # establece la semilla aleatoria para generar el conjunto de datos.
)

In [ ]:
"""
Se definen diferentes configuraciones de RandomForestClassifier con diferentes valores 
para el parámetro max_features y se almacenan en una lista llamada ensemble_clfs.
"""

ensemble_clfs = [
    (
        "RandomForestClassifier, max_features='sqrt'",
        RandomForestClassifier(
            warm_start=True,
            oob_score=True,
            max_features="sqrt",
            random_state=RANDOM_STATE,
        ),
    ),
    (
        "RandomForestClassifier, max_features='log2'",
        RandomForestClassifier(
            warm_start=True,
            max_features="log2",
            oob_score=True,
            random_state=RANDOM_STATE,
        ),
    ),
    (
        "RandomForestClassifier, max_features=None",
        RandomForestClassifier(
            warm_start=True,
            max_features=None,
            oob_score=True,
            random_state=RANDOM_STATE,
        ),
    ),
]

In [ ]:
# NOTE: Setting the `warm_start` construction parameter to `True` disables
# support for parallelized ensembles but is necessary for tracking the OOB
# error trajectory during training.

In [ ]:
# max_features --> especifica cómo se seleccionan las características aleatorias

# none -> (valor predeterminado), todas las características se considerarán para cada división en cada nodo del árbol. 
# sqrt -> el modelo considerará la raíz cuadrada del número total de características (25) en el conjunto de datos para cada división.
# log2 -> el modelo considerará el logaritmo base 2 del número total de características en el conjunto de datos para cada división.


In [ ]:
# Cuando oob_score = True, el modelo de Bosques Aleatorios calculará automáticamente el puntaje de error fuera 
# de la bolsa después del entrenamiento. 
# Este puntaje OOB proporciona una estimación del rendimiento del modelo en datos no vistos 
# sin necesidad de un conjunto de validación separado.

# Al establecer oob_score=True, se calcula y se proporciona un puntaje de error fuera de la bolsa 
# después del entrenamiento del modelo de Bosques Aleatorios, lo que puede ser útil para evaluar su rendimiento de generalización.

In [ ]:
""" 
Se crea un diccionario llamado error_rate que mapea el nombre del clasificador a una lista vacía. 
Esta lista almacenará las tasas de error fuera de la bolsa (OOB) para cada configuración de clasificador. 
"""

error_rate = OrderedDict((label, []) for label, _ in ensemble_clfs)
# Mapea un name classificador a una lista de (<n_estimators>, <error rate>) pares.
# orderedDict -> Dictionary that remembers insertion order

In [ ]:
""" 
crea un diccionario llamado error_rate, donde las claves son los nombres de los clasificadores y los valores son listas vacías. 
    · orderedDict -> Dictionary that remembers insertion order
    · (label, []): Aquí, para cada elemento en ensemble_clfs, estamos creando una tupla que consta de dos partes:
        label: Es el nombre del clasificador en ensemble_clfs.
        []: Es una lista vacía.
    · for label, _ in ensemble_clfs: Esto itera sobre cada elemento en ensemble_clfs, que es una lista de tuplas donde 
    cada tupla contiene el nombre de un clasificador y el clasificador mismo. El _ se utiliza para ignorar la parte 
    del clasificador en cada tupla, ya que en este caso solo estamos interesados en los nombres.
"""

In [ ]:
# Se define un rango de valores para n_estimators que se explorarán
min_estimators = 15
max_estimators = 150

In [ ]:
for label, clf in ensemble_clfs:  # Iteración sobre ensemble_clfs
    for i in range(min_estimators, max_estimators + 1, 5):  # Iteración sobre el rango de estimadores
# Estamos iterando sobre un rango de estimadores. Comenzamos desde min_estimators y vamos en incrementos de 5 hasta max_estimators.
        clf.set_params(n_estimators=i)
        clf.fit(X, y)

In [ ]:
"""
label -> nombre clasificador
clf -> clasificador
Ajustamos (fit()) el clasificador al conjunto de datos de entrenamiento (X, y). 
Esto significa que estamos entrenando el clasificador con un número específico de estimadores en cada iteración.
"""

In [ ]:
 # Cálculo del error fuera de la bolsa (OOB):
        oob_error = 1 - clf.oob_score_
    # Almacenamiento del error OOB en error_rate:
        error_rate[label].append((i, oob_error))

In [ ]:
"""
Una vez que el clasificador está ajustado, calculamos el error fuera de la bolsa (OOB) 
utilizando el atributo oob_score_ del clasificador. Este atributo proporciona la puntuación OOB del modelo, 
que es una medida de su rendimiento en datos no vistos.

Finalmente, almacenamos el error OOB para el clasificador actual en el diccionario error_rate. 
El error OOB se almacena como una tupla (n_estimators, oob_error) dentro de una lista asociada con la 
etiqueta del clasificador en el diccionario error_rate.
"""

In [ ]:
# Generate the "OOB error rate" vs. "n_estimators" plot.
for label, clf_err in error_rate.items():
    xs, ys = zip(*clf_err)
    plt.plot(xs, ys, label=label)

 xs, ys = zip(*clf_err) --> Desempaquetado de las listas de tuplas:
La función zip(*clf_err) toma la lista de tuplas clf_err y la desempaqueta en dos listas separadas xs y ys. Esto significa que xs contendrá todos los valores de n_estimators y ys contendrá todos los valores de oob_error para el clasificador actual.
 